In [1]:
import pandas as pd
import numpy as np
import holidays
from sklearn.preprocessing import LabelEncoder

In [ ]:
# NOTE: Custome Module !!
from module.dataframe import df_query_by_type, series_string_filter
from module.preprocess import do_preprocessing

---

In [4]:
train = pd.read_parquet("./database/train.parquet")
test = pd.read_parquet("./database/test.parquet")

In [5]:
train, test = do_preprocessing(train, test)

In [41]:
train_x, train_y = train.drop(columns="target"), train["target"]
test_x = test

---

In [42]:
# MAX FEATURE!!!
CAT = ["day_of_week",
"weight_restricted",
"road_type",
"start_turn_restricted",
"end_turn_restricted",
"is_holiday",]
train_x = pd.get_dummies(train_x,columns=CAT)
test_x = pd.get_dummies(test_x,columns=CAT)

---

In [43]:
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [44]:
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)
eval_set = [(x_valid, y_valid)]

lgb_params = {
    "objective": "regression",
    "boosting_type": "gbdt",
    "metric": "rmse",
    "n_estimators": 800,
    "learning_rate": 0.05,
    "feature_fraction": 0.9,
    "reg_alpha": 0.1,
    "reg_lambda": 0.2,
    "subsample": 0.7,
    "colsample_bytree": 0.6,
    "verbose": 1,
}
lgb_regress = LGBMRegressor(
    **lgb_params,
)

lgb_regress.fit(x_train, y_train, eval_metric="rmse", eval_set=eval_set)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.090791 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 127
[LightGBM] [Info] Number of data points in the train set: 3760973, number of used features: 28
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Start training from score 42.786869


LGBMRegressor(colsample_bytree=0.6, feature_fraction=0.9, learning_rate=0.05,
              metric='rmse', n_estimators=800, objective='regression',
              reg_alpha=0.1, reg_lambda=0.2, subsample=0.7, verbose=1)

In [45]:
# Eval
from sklearn.metrics import r2_score as R2
from sklearn.metrics import root_mean_squared_error as RMSE
from sklearn.metrics import median_absolute_error as MAE

MAE(lgb_regress.predict(x_valid), y_valid), R2(lgb_regress.predict(x_valid), y_valid),RMSE(lgb_regress.predict(x_valid), y_valid)

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9


(5.844639289551871, 0.4139352572095595, 9.642905821619575)

## Submission
---

In [46]:
sample_submission = pd.read_csv('./database/sample_submission.csv')

In [47]:
sample_submission["target"] = lgb_regress.predict(test_x)
sample_submission.to_csv("./submit.csv", index = False)
sample_submission

[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=0.6 will be ignored. Current value: feature_fraction=0.9


,id,target
0,TEST_000000,23.661287
1,TEST_000001,46.239807
2,TEST_000002,50.380547
3,TEST_000003,37.302664
4,TEST_000004,39.573109
...,...,...
291236,TEST_291236,48.228557
291237,TEST_291237,38.301339
291238,TEST_291238,22.812145
291239,TEST_291239,26.532071


![firstsubmission](./0003.png)